In [43]:
import pandas as pd
import numpy as np

## Step 1
Load the "messy dataset" and explore the Data – Import the dataset into Phyton. Explore: How many rows and columns? Are their any potential problematic variables?

In [44]:
messy = pd.read_csv("/Users/hannahw/DTSC 3601/messy_dataset.csv")
messy

,CustomerID,Age,State,Income,Purchases,LastPurchaseDate,Review,Churn
0,CUST001,40.0,CA,40000.0,0.0,NaN,NaN,NaN
1,CUST002,40.0,california,40000.0,1.0,NaN,NaN,NaN
2,CUST003,60.0,NY,40000.0,10.0,2021-13-40,NaN,N
3,CUST004,200.0,California,1000000.0,10.0,2021-13-40,Great product!,Y
4,CUST005,200.0,CA,40000.0,NaN,5/20/2021,Loved it!!!,Yes
...,...,...,...,...,...,...,...,...
95,CUST096,30.0,California,70000.0,10.0,2021-13-40,Ok,N
96,CUST097,25.0,CA,50000.0,NaN,2021-13-40,would buy again,NaN
97,CUST098,25.0,NaN,1000000.0,1.0,NaN,Great product!,N
98,CUST099,25.0,California,50000.0,3.0,NaN,Great product!,Y


There are 100 rows and 8 columns. Age, State, Churn variables look problematic due to inconsistent abbreviatoins and invalid data (ex: -5 for age).

## Step 2 

Identify columns with missing values and Decide: How to handle them? (e.g. drop the value, mean/median, mode, forward fill, etc)

The columns with missing values are Purchases, LastPurchaseDate, Review, and Churn.

In [ ]:
"""using assumption that customers who have purchased 
multiple times are less likely to churn to fill 
missing values """
messy.loc[messy['Churn'].isna() & (messy['Purchases'] > 1.0), 'Churn'] = 'N'
messy.loc[messy['Churn'].isna() & (messy['Purchases'] <= 1.0), 'Churn'] = 'Y'
messy.loc[messy['Churn'].isna() & (messy['Purchases'].isna()), 'Churn'] = 'Y'  # or 'Unknown'

# Since review is NaN, mark as No Review 
messy['Review'] = messy['Review'].fillna("No Review")

# since all purchases are in 2021, make 2021-1-1 default
messy['LastPurchaseDate'] = messy['LastPurchaseDate'].fillna("2021-1-1")

# assuming that if on  
messy['Purchases'] = messy['Purchases'].fillna(messy['Purchases'].median())

messy

,CustomerID,Age,State,Income,Purchases,LastPurchaseDate,Review,Churn
0,CUST001,40.0,CA,40000.0,0.0,2021-1-1,No Review,Y
1,CUST002,40.0,california,40000.0,1.0,2021-1-1,No Review,Y
2,CUST003,60.0,NY,40000.0,10.0,2021-13-40,No Review,N
3,CUST004,200.0,California,1000000.0,10.0,2021-13-40,Great product!,Y
4,CUST005,200.0,CA,40000.0,2.0,5/20/2021,Loved it!!!,Yes
...,...,...,...,...,...,...,...,...
95,CUST096,30.0,California,70000.0,10.0,2021-13-40,Ok,N
96,CUST097,25.0,CA,50000.0,2.0,2021-13-40,would buy again,Y
97,CUST098,25.0,NaN,1000000.0,1.0,2021-1-1,Great product!,N
98,CUST099,25.0,California,50000.0,3.0,2021-1-1,Great product!,Y


## Step 3 

Identify Invalid or Inconsistent Entries and Fix: The following variables: Age, State, Churn (e.g. Yes/No)

In [46]:
# fixing Age column by coverting to numeric, and replacing invalid or missing values with the median
messy['Age'] = pd.to_numeric(messy['Age'], errors='coerce')
messy.loc[(messy['Age'] < 0) | (messy['Age'] > 120), 'Age'] = np.nan
messy['Age'].fillna(messy['Age'].median(), inplace=True)

# unifying state variations
messy['State'] = messy['State'].str.upper()

state_map = {
    'N.Y.': 'NY',
    'NEW YORK': 'NY',
    'CALIFORNIA': 'CA'
}
messy['State'] = messy['State'].replace(state_map)
# replace NaN with mode
messy["State"].fillna(messy["State"].mode()[0], inplace=True)

# unifying churn
messy['Churn'] = messy['Churn'].str.upper()

churn_map = {
    'Y': 'YES', 'YES': 'YES',
    'N': 'NO',  'NO': 'NO',
}
messy['Churn'] = messy['Churn'].map(churn_map)

messy['Churn'].fillna('Unknown', inplace=True)

messy

,CustomerID,Age,State,Income,Purchases,LastPurchaseDate,Review,Churn
0,CUST001,40.0,CA,40000.0,0.0,2021-1-1,No Review,YES
1,CUST002,40.0,CA,40000.0,1.0,2021-1-1,No Review,YES
2,CUST003,60.0,NY,40000.0,10.0,2021-13-40,No Review,NO
3,CUST004,45.0,CA,1000000.0,10.0,2021-13-40,Great product!,YES
4,CUST005,45.0,CA,40000.0,2.0,5/20/2021,Loved it!!!,YES
...,...,...,...,...,...,...,...,...
95,CUST096,30.0,CA,70000.0,10.0,2021-13-40,Ok,NO
96,CUST097,25.0,CA,50000.0,2.0,2021-13-40,would buy again,YES
97,CUST098,25.0,CA,1000000.0,1.0,2021-1-1,Great product!,NO
98,CUST099,25.0,CA,50000.0,3.0,2021-1-1,Great product!,YES


## Step 4

Detect and Handle Outliers – Inspect Income and Decide how to handle outliers.

In [ ]:
# using IQR to determine upper/lower bounds to check for outliers and replace them
Q1 = messy['Income'].quantile(0.25)
Q3 = messy['Income'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

messy = messy[(messy['Income'] >= lower_bound) & (messy['Income'] <= upper_bound)]

messy

,CustomerID,Age,State,Income,Purchases,LastPurchaseDate,Review,Churn
0,CUST001,40.0,CA,40000.0,0.0,2021-1-1,No Review,YES
1,CUST002,40.0,CA,40000.0,1.0,2021-1-1,No Review,YES
2,CUST003,60.0,NY,40000.0,10.0,2021-13-40,No Review,NO
3,CUST004,45.0,CA,1000000.0,10.0,2021-13-40,Great product!,YES
4,CUST005,45.0,CA,40000.0,2.0,5/20/2021,Loved it!!!,YES
...,...,...,...,...,...,...,...,...
95,CUST096,30.0,CA,70000.0,10.0,2021-13-40,Ok,NO
96,CUST097,25.0,CA,50000.0,2.0,2021-13-40,would buy again,YES
97,CUST098,25.0,CA,1000000.0,1.0,2021-1-1,Great product!,NO
98,CUST099,25.0,CA,50000.0,3.0,2021-1-1,Great product!,YES


## Step 5
Working with Dates – Check LastPurchaseDate column and Convert valid dates to daytime format.


In [ ]:
# since there are multiple date types, try converting each
def parse_date(x):
    try:
        return pd.to_datetime(x, format="%Y-%m-%d", errors="raise")
    except:
        try:
            return pd.to_datetime(x, format="%m/%d/%Y", errors="raise")
        except:
            return pd.NaT
messy["LastPurchaseDate"] = messy["LastPurchaseDate"].apply(parse_date)

# fill the rest of the values in with jan 1, 2021 since all the values are in 2021
messy['LastPurchaseDate'] = messy['LastPurchaseDate'].fillna("2021-1-1")
messy


/var/folders/wj/k4ch97vx6pvbff2shzf1g9_80000gn/T/ipykernel_21384/1898402588.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messy["LastPurchaseDate"] = messy["LastPurchaseDate"].apply(parse_date)
/var/folders/wj/k4ch97vx6pvbff2shzf1g9_80000gn/T/ipykernel_21384/1898402588.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messy['LastPurchaseDate'] = messy['LastPurchaseDate'].fillna("2021-1-1")


,CustomerID,Age,State,Income,Purchases,LastPurchaseDate,Review,Churn
0,CUST001,40.0,CA,40000.0,0.0,2021-01-01,No Review,YES
1,CUST002,40.0,CA,40000.0,1.0,2021-01-01,No Review,YES
2,CUST003,60.0,NY,40000.0,10.0,2021-01-01,No Review,NO
3,CUST004,45.0,CA,1000000.0,10.0,2021-01-01,Great product!,YES
4,CUST005,45.0,CA,40000.0,2.0,2021-05-20,Loved it!!!,YES
...,...,...,...,...,...,...,...,...
95,CUST096,30.0,CA,70000.0,10.0,2021-01-01,Ok,NO
96,CUST097,25.0,CA,50000.0,2.0,2021-01-01,would buy again,YES
97,CUST098,25.0,CA,1000000.0,1.0,2021-01-01,Great product!,NO
98,CUST099,25.0,CA,50000.0,3.0,2021-01-01,Great product!,YES


## Step 6 
Explore the review column and Create a new binary feature called “PositiveReview” . If the answer contains “great”, “love”, “buy again” then assign =1, everything else assign=0.

In [49]:
messy["PositiveReview"] = (
    messy["Review"]
    .astype(str) 
    .str.contains("great|love|buy again", case=False, na=False)
    .astype(int)
)

messy

/var/folders/wj/k4ch97vx6pvbff2shzf1g9_80000gn/T/ipykernel_21384/3087975012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messy["PositiveReview"] = (


,CustomerID,Age,State,Income,Purchases,LastPurchaseDate,Review,Churn,PositiveReview
0,CUST001,40.0,CA,40000.0,0.0,2021-01-01,No Review,YES,0
1,CUST002,40.0,CA,40000.0,1.0,2021-01-01,No Review,YES,0
2,CUST003,60.0,NY,40000.0,10.0,2021-01-01,No Review,NO,0
3,CUST004,45.0,CA,1000000.0,10.0,2021-01-01,Great product!,YES,1
4,CUST005,45.0,CA,40000.0,2.0,2021-05-20,Loved it!!!,YES,1
...,...,...,...,...,...,...,...,...,...
95,CUST096,30.0,CA,70000.0,10.0,2021-01-01,Ok,NO,0
96,CUST097,25.0,CA,50000.0,2.0,2021-01-01,would buy again,YES,1
97,CUST098,25.0,CA,1000000.0,1.0,2021-01-01,Great product!,NO,1
98,CUST099,25.0,CA,50000.0,3.0,2021-01-01,Great product!,YES,1


## Step 7
Save the cleaned dataset and Submit your Jupyter notebook with the work via Canvas

In [50]:
# mac and cheese